# Loading data

In [1]:
using JuMP
using CSV
using DataFrames
using HiGHS
using PlotlyJS
using StatsBase
using statistics

ArgumentError: ArgumentError: Package statistics not found in current path.
- Run `import Pkg; Pkg.add("statistics")` to install the statistics package.

In [2]:
lines = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/lines.csv", DataFrame)
demand = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/demand.csv", DataFrame)
busses = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/busses.csv", DataFrame)
bus_line_stops = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/bus-lines.csv", DataFrame)

Row,bus_line_id,stop_ids,stop_x,stop_y
,Int64,Int64,Float64,Float64
1,1,1,2.2,36.8
2,1,2,4.4,40.1
3,1,3,12.5,42.7
4,1,4,20.2,44.0
5,2,1,40.9,48.2
6,2,2,46.2,40.7
7,2,3,56.6,38.2
8,3,1,44.6,15.2
9,3,2,52.0,20.4


### Bus line stops

In [3]:
bus_line_stops

Row,bus_line_id,stop_ids,stop_x,stop_y
,Int64,Int64,Float64,Float64
1,1,1,2.2,36.8
2,1,2,4.4,40.1
3,1,3,12.5,42.7
4,1,4,20.2,44.0
5,2,1,40.9,48.2
6,2,2,46.2,40.7
7,2,3,56.6,38.2
8,3,1,44.6,15.2
9,3,2,52.0,20.4


### Lines

In [4]:
lines

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,1,1,20.0
2,2,1,40.0
3,3,1,60.0
4,1,2,20.0
5,2,2,35.0
6,3,2,50.0
7,1,3,16.0
8,2,3,32.0
9,3,3,48.0


### Demand

In [5]:
demand

Row,demand_id,line_id,bus_line_id,origin_stop_id,destination_stop_id,demand
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,4,1
2,2,1,1,2,3,2
3,3,1,1,2,4,1
4,4,2,1,3,4,1
5,5,2,1,1,3,2
6,6,1,2,1,2,4
7,7,3,2,1,3,1
8,8,1,3,1,6,2
9,9,1,3,2,5,1


### Busses

In [6]:
busses

Row,bus_id,capacity,shift_start,break_start_1,break_end_1,break_start_2,break_end_2,shift_end
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,10,0,0,0,30,40,70
2,2,10,5,35,40,50,55,75
3,3,10,10,10,10,40,50,80
4,4,10,10,55,60,75,80,90
5,5,10,0,20,30,50,60,90
6,6,10,0,0,0,45,55,75
7,7,10,0,25,35,55,65,90
8,8,10,30,45,55,75,85,100
9,9,10,0,20,30,50,55,100


# Data understanding

## Looking at the stops

In [7]:
using PlotlyJS
using DataFrames
using Statistics

# Gruppiere nach bus_line_id
grouped_data = groupby(bus_line_stops, :bus_line_id)

# Sammle alle Traces
traces = PlotlyJS.AbstractTrace[]  # use correct type

# Linien- und Beschriftungstraces erzeugen
for grp in grouped_data
    sort!(grp, :stop_ids)

    # Linien-Trace mit Markern
    push!(traces, scatter(
        x = grp.stop_x,
        y = grp.stop_y,
        mode = "lines+markers+text",
        name = "Linie $(unique(grp.bus_line_id)[1])",
        marker = attr(size = 8),
        text = string.(grp.stop_ids),
        textposition = "top center",
        textfont = attr(color="black", size=10)
    ))

    # Distanzen zwischen benachbarten Haltestellen
    for i in 1:(nrow(grp) - 1)
        x1, y1 = grp.stop_x[i], grp.stop_y[i]
        x2, y2 = grp.stop_x[i+1], grp.stop_y[i+1]
        dist = round(sqrt((x2 - x1)^2 + (y2 - y1)^2), digits=2)
        mx, my = mean([x1, x2]), mean([y1, y2])

        # Distanztext hinzufügen
        push!(traces, scatter(
            x = [mx], y = [my],
            mode = "text",
            text = ["d = $dist"],
            textposition = "bottom center",
            textfont = attr(size = 9, color = "gray"),
            showlegend = false
        ))
    end
end

# Layout festlegen
layout = Layout(
    title = "Buslinien und ihre Haltestellen",
    xaxis_title = "X-Koordinate",
    yaxis_title = "Y-Koordinate",
    legend = attr(x=1.02, y=1)
)

# Plot erstellen
p = Plot(traces, layout)

# In HTML-Datei speichern (im gleichen Ordner wie Code)
PlotlyJS.savefig(p, "/Users/alexanderklaus/Desktop/Masterthesis/Code/output/buslinien_plot.html")


plot_initial_stops (generic function with 1 method)

# Preprocessing

## Input settings

### Adding the depot

In [9]:
depot=(bus_line_id=0,stop_ids=0,stop_x=30,stop_y=30)
insert!(bus_line_stops,1,depot)
depot=(line_id=0,bus_line_id=0, start_time=0)
insert!(lines,1,depot)

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,0,0,0.0
2,1,1,20.0
3,2,1,40.0
4,3,1,60.0
5,1,2,20.0
6,2,2,35.0
7,3,2,50.0
8,1,3,16.0
9,2,3,32.0


### Bus velocity

In [10]:
bus_velocity = 100 #in km/h

100

### Break Time

In [11]:
break_time= "break1" #oder "break1"

"break1"

## Processing data

### Customer trips

In [12]:
customer_trips = [((r.bus_line_id, r.line_id, r.origin_stop_id), (r.bus_line_id, r.line_id, r.destination_stop_id)) for r in eachrow(demand)]

17-element Vector{Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}}:
 ((1, 1, 1), (1, 1, 4))
 ((1, 1, 2), (1, 1, 3))
 ((1, 1, 2), (1, 1, 4))
 ((1, 2, 3), (1, 2, 4))
 ((1, 2, 1), (1, 2, 3))
 ((2, 1, 1), (2, 1, 2))
 ((2, 3, 1), (2, 3, 3))
 ((3, 1, 1), (3, 1, 6))
 ((3, 1, 2), (3, 1, 5))
 ((3, 2, 1), (3, 2, 2))
 ((3, 2, 4), (3, 2, 6))
 ((4, 2, 1), (4, 2, 2))
 ((4, 4, 1), (4, 4, 2))
 ((5, 1, 3), (5, 1, 5))
 ((5, 3, 2), (5, 3, 4))
 ((5, 3, 2), (5, 3, 3))
 ((5, 3, 1), (5, 3, 4))

### Creating nodes for all stops, on every busline, for every tour

In [13]:
nodes = []

for r in eachrow(lines)
    for row in eachrow(bus_line_stops)
        if r.bus_line_id == row.bus_line_id
            start_time_temp=0.0
            if row.stop_ids == 1
                start_time_temp = r.start_time
            else
                # alle Stopps dieser Linie sortieren
                stops = sort(filter(r2 -> r2.bus_line_id == row.bus_line_id, eachrow(bus_line_stops)), by = r2 -> r2.stop_ids)
                
                # Travel time vom ersten Knoten bis aktuellen Knoten:
                total_distance = 0.0
                for k in 1:(row.stop_ids - 1)
                    x1, y1 = stops[k].stop_x, stops[k].stop_y
                    x2, y2 = stops[k+1].stop_x, stops[k+1].stop_y
                    total_distance += sqrt((x2 - x1)^2 + (y2 - y1)^2)
                end
                travel_time = (total_distance / bus_velocity) * 60  # Minuten

                # Startzeit = Startzeit erster Knoten + Travel Time
                start_time_temp = r.start_time + travel_time
            end
            push!(nodes, (
                bus_line_id = row.bus_line_id,
                line_id     = r.line_id,
                stop_id     = row.stop_ids,
                coord_x     = row.stop_x,
                coord_y     = row.stop_y,
                start_time  = start_time_temp
            ))
                
        end
    end
end

In [14]:
print("Nodes:")
nodes

Nodes:

68-element Vector{Any}:
 (bus_line_id = 0, line_id = 0, stop_id = 0, coord_x = 30.0, coord_y = 30.0, start_time = 0.0)
 (bus_line_id = 1, line_id = 1, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 20.0)
 (bus_line_id = 1, line_id = 1, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 22.379663841806234)
 (bus_line_id = 1, line_id = 1, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 27.483897378783162)
 (bus_line_id = 1, line_id = 1, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 32.16927890006828)
 (bus_line_id = 1, line_id = 2, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 40.0)
 (bus_line_id = 1, line_id = 2, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 42.379663841806234)
 (bus_line_id = 1, line_id = 2, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 47.48389737878316)
 (bus_line_id = 1, line_id = 2, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 52.16927890006828)
 (bus_line_id = 1, line_id = 3, stop

### Creating connections

In [15]:
function eucleadian_distance(x1, y1, x2, y2)
    return sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

eucleadian_distance (generic function with 1 method)

In [16]:
# Define a struct to hold the travel_time and end_time
struct Connection
    distance::Float64
    travel_time::Float64
    end_time::Float64
end

# Update the dictionary to use the struct as the value type
connections_dict = Dict()

for node1 in nodes
    for node2 in nodes
        if node1 != node2
            #skip if the nodes are the same
            origin_node = (node1.bus_line_id, node1.line_id, node1.stop_id)
            goal_node = (node2.bus_line_id, node2.line_id, node2.stop_id)
            if node1.bus_line_id == node2.bus_line_id
                if node1.line_id == node2.line_id
                    if node1.stop_id == node2.stop_id
                        # Nodes are the same
                        break
                    else
                        # Nodes are on the same tour
                        stops = sort(filter(n -> n.bus_line_id == node1.bus_line_id && n.line_id==node1.line_id, nodes), by = n -> n.stop_id)
                        start_index = findfirst(n -> n.stop_id == node1.stop_id, stops)
                        end_index = findfirst(n -> n.stop_id == node2.stop_id, stops)            
                        if start_index < end_index
                            #if origin node comes before goal node
                            total_distance = 0.0
                            for i in start_index:(end_index - 1)
                                x1, y1 = stops[i].coord_x, stops[i].coord_y
                                x2, y2 = stops[i + 1].coord_x, stops[i + 1].coord_y
                                total_distance += eucleadian_distance(x1, y1, x2, y2)
                            end
                            distance= total_distance
                            travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                            end_time = node1.start_time + travel_time
                            connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                        elseif start_index > end_index
                            #=
                            total_distance = 0.0
                            for i in start_index:(end_index - 1)
                                x1, y1 = stops[i].coord_x, stops[i].coord_y
                                x2, y2 = stops[i + 1].coord_x, stops[i + 1].coord_y
                                total_distance += eucleadian_distance(x1, y1, x2, y2)
                            end
                            distance= total_distance
                            =#
                            distance=eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                            travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                            end_time = node1.start_time + travel_time
                            connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                        end
                    end
                else
                    #nodes are on the same bus line but not the same tour
                    distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                    travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                    end_time = node1.start_time + travel_time
                    connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                end
            else
                # Nodes are on different bus lines
                distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                end_time = node1.start_time + travel_time
                connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
            end
        end
    end
end

println("Connections dictionary:\n(origin_node, goal_node) -> (distance, travel_time, end_time)")
sort(collect(connections_dict), by = x -> x.first)

Connections dictionary:
(origin_node, goal_node) -> (distance, travel_time, end_time)


4556-element Vector{Pair{Any, Any}}:
 ((0, 0, 0), (1, 1, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 1, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213661408335)
 ((0, 0, 0), (1, 1, 3)) => Connection(21.622673285234647, 12.973603971140788, 12.973603971140788)
 ((0, 0, 0), (1, 1, 4)) => Connection(17.089177862027185, 10.253506717216311, 10.253506717216311)
 ((0, 0, 0), (1, 2, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 2, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213661408335)
 ((0, 0, 0), (1, 2, 3)) => Connection(21.622673285234647, 12.973603971140788, 12.973603971140788)
 ((0, 0, 0), (1, 2, 4)) => Connection(17.089177862027185, 10.253506717216311, 10.253506717216311)
 ((0, 0, 0), (1, 3, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 3, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213

In [17]:
connections_dict[(0,0,0),(1,1,1)]

Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)

### Dictionary mit Stopp-IDs je Linie

In [18]:
line_stops_dict = Dict{Int, Vector{Int}}()

for row in eachrow(bus_line_stops)
    l = row.bus_line_id
    stop = row.stop_ids
    # Depot-Zeile überspringen
    if l == 0 && stop == 0
        continue
    end
    if haskey(line_stops_dict, l)
        push!(line_stops_dict[l], stop)
    else
        line_stops_dict[l] = [stop]
    end
end
println("Dictionary of lines with their respective stop-sequences:")
sort(line_stops_dict)

Dictionary of lines with their respective stop-sequences:


OrderedCollections.OrderedDict{Int64, Vector{Int64}} with 5 entries:
  1 => [1, 2, 3, 4]
  2 => [1, 2, 3]
  3 => [1, 2, 3, 4, 5, 6]
  4 => [1, 2]
  5 => [1, 2, 3, 4, 5]

### Populating set A

#### Depot & O-D connections

In [19]:
A_set = Set{Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}}()
depot = (0, 0, 0)
for row in eachrow(demand)
    from_node = (row.bus_line_id, row.line_id, row.origin_stop_id)
    to_node   = (row.bus_line_id, row.line_id, row.destination_stop_id)
    trip_key  = (from_node, to_node)
    idx_origin = findfirst(x -> x.bus_line_id == from_node[1] && x.line_id == from_node[2] && x.stop_id == from_node[3], nodes)
    
    t_start_origin_tour = nodes[idx_origin].start_time
    t_depot_origin_stop = connections_dict[(depot,from_node)].travel_time
    
    if t_depot_origin_stop < t_start_origin_tour
        # Depot -> origin
        if haskey(connections_dict, (depot, from_node))
            push!(A_set, (depot, from_node))
        else
            display("WARNUNG - Verbindung fehlt in connections_dict: ", (depot, from_node))
        end
        # destination -> Depot
        if haskey(connections_dict, (to_node, depot))
            push!(A_set, (to_node, depot))
        else
            display("WARNUNG - Verbindung fehlt in connections_dict: ", (to_node, depot))
        end
        # origin -> destination
        if !(trip_key in A_set)
            if haskey(connections_dict, trip_key)
                push!(A_set, trip_key)
            else
                display("WARNUNG - Verbindung fehlt in connections_dict: ", trip_key)
            end
        else
            display(" Verbindung $trip_key bereits in A_set.")
        end
    else
        display("Tour $trip_key kann nicht rechtzeitig gestartet werden, da die Reisezeit zwischen Depot und dem Start-Knoten der Tour zu lang ist.")
    end
end


In [20]:
sorted_A=sort(collect(A_set))
println("\n\n A_set:")
for a in sorted_A
    println(a)
end



 A_set:
((0, 0, 0), (1, 1, 1))
((0, 0, 0), (1, 1, 2))
((0, 0, 0), (1, 2, 1))
((0, 0, 0), (1, 2, 3))
((0, 0, 0), (2, 1, 1))
((0, 0, 0), (2, 3, 1))
((0, 0, 0), (3, 1, 1))
((0, 0, 0), (3, 1, 2))
((0, 0, 0), (3, 2, 1))
((0, 0, 0), (3, 2, 4))
((0, 0, 0), (4, 2, 1))
((0, 0, 0), (4, 4, 1))
((0, 0, 0), (5, 1, 3))
((0, 0, 0), (5, 3, 1))
((0, 0, 0), (5, 3, 2))
((1, 1, 1), (1, 1, 4))
((1, 1, 2), (1, 1, 3))
((1, 1, 2), (1, 1, 4))
((1, 1, 3), (0, 0, 0))
((1, 1, 4), (0, 0, 0))
((1, 2, 1), (1, 2, 3))
((1, 2, 3), (0, 0, 0))
((1, 2, 3), (1, 2, 4))
((1, 2, 4), (0, 0, 0))
((2, 1, 1), (2, 1, 2))
((2, 1, 2), (0, 0, 0))
((2, 3, 1), (2, 3, 3))
((2, 3, 3), (0, 0, 0))
((3, 1, 1), (3, 1, 6))
((3, 1, 2), (3, 1, 5))
((3, 1, 5), (0, 0, 0))
((3, 1, 6), (0, 0, 0))
((3, 2, 1), (3, 2, 2))
((3, 2, 2), (0, 0, 0))
((3, 2, 4), (3, 2, 6))
((3, 2, 6), (0, 0, 0))
((4, 2, 1), (4, 2, 2))
((4, 2, 2), (0, 0, 0))
((4, 4, 1), (4, 4, 2))
((4, 4, 2), (0, 0, 0))
((5, 1, 3), (5, 1, 5))
((5, 1, 5), (0, 0, 0))
((5, 3, 1), (5, 3, 4))
(

#### D-O connections

In [21]:
missing_trips_in_connections_dict =[]
for i in 1:nrow(demand)
    row1 = demand[i, :]
    from_node_orig = (row1.bus_line_id, row1.line_id, row1.origin_stop_id)
    from_node_dest = (row1.bus_line_id, row1.line_id, row1.destination_stop_id)
    #println("\n\nChecking connections for trip from $from_node_dest to $depot...")
    if (from_node_dest, depot) in A_set
        #println("  Connection from $from_node_dest to $depot exists in A_set.")
        for j in 1:nrow(demand)
            if i == j
                continue
            end
            row2 = demand[j, :]
            to_node = (row2.bus_line_id, row2.line_id, row2.origin_stop_id)
            trip_key = (from_node_dest, to_node)
            #println("   Checking if connection $from_node_dest to $to_node in A_set...")
            if !(trip_key in A_set) 
                if haskey(connections_dict, (from_node_dest, to_node))
                    t_end_origin_line = connections_dict[(from_node_orig, from_node_dest)].end_time
                    t_between_lines = connections_dict[(from_node_dest, to_node)].travel_time
                    idx_goal = findfirst(x -> x.bus_line_id == to_node[1] && x.line_id == to_node[2] && x.stop_id == to_node[3], nodes)
                    t_start_second_tour = nodes[idx_goal].start_time
                    #println("    Checking if time constraint holds for connection $trip_key...")
                    if t_end_origin_line + t_between_lines <= t_start_second_tour
                        #prinln("     Added connection $trip_key, because end of tour $from_node_dest is $t_end_origin_line and start of tour $to_node is $t_start_second_tour.")
                        if !(trip_key in A_set)
                            push!(A_set, trip_key)
                            println("Added connection: ", trip_key)
                        end
                    else
                        #println("      Time constraint does not hold for connection $trip_key.")
                        continue
                    end
                else 
                    println("WARNUNG - Verbindung $trip_key fehlt in connections_dict.")
                    push!(missing_trips_in_connections_dict, trip_key)
                end
            else
                println("Connection $trip_key already in A_set.")
            end
        end
    else
        continue
    end
end


Added connection: ((1, 1, 4), (1, 2, 3))
Added connection: ((1, 1, 4), (2, 3, 1))
Added connection: ((1, 1, 4), (3, 2, 4))
Added connection: ((1, 1, 4), (4, 4, 1))
Added connection: ((1, 1, 3), (1, 2, 3))
Added connection: ((1, 1, 3), (1, 2, 1))
Added connection: ((1, 1, 3), (2, 3, 1))
Added connection: ((1, 1, 3), (3, 2, 4))
Added connection: ((1, 1, 3), (4, 4, 1))
Added connection: ((1, 1, 3), (5, 3, 2))
Connection ((1, 1, 3), (5, 3, 2)) already in A_set.
Added connection: ((1, 1, 3), (5, 3, 1))
Connection ((1, 1, 4), (1, 2, 3)) already in A_set.
Connection ((1, 1, 4), (2, 3, 1)) already in A_set.
Connection ((1, 1, 4), (3, 2, 4)) already in A_set.
Connection ((1, 1, 4), (4, 4, 1)) already in A_set.
WARNUNG - Verbindung ((1, 2, 3), (1, 2, 3)) fehlt in connections_dict.
Added connection: ((2, 1, 2), (1, 2, 3))
Added connection: ((2, 1, 2), (2, 3, 1))
Added connection: ((2, 1, 2), (3, 2, 4))
Added connection: ((2, 1, 2), (4, 4, 1))
Added connection: ((2, 1, 2), (5, 3, 2))
Connection ((

#### Backwards connections between overlapping trips

In [22]:
for i in 1:nrow(demand)
    trip1= demand[i, :]
    key1 = (trip1.bus_line_id, trip1.line_id)
    for j in 1:nrow(demand)
        trip2 = demand[j, :]
        key2 = (trip2.bus_line_id, trip2.line_id)
        if trip1 != trip2 && key1==key2
            s_k= trip1.origin_stop_id
            s_i= trip1.destination_stop_id
            s_j= trip2.origin_stop_id
            s_h= trip2.destination_stop_id
            #println("\nFor line $key1: Checking tours (k:$s_k - i:$s_i) and (j:$s_j - h:$s_h)...")
            if s_i > s_j && s_k < s_j < s_i && s_h > s_j && trip1.demand > 0 && trip2.demand >0
                #print("k: $s_k, j: $s_j,  i: $s_i, h: $s_h, demand from k to i: $(trip1.demand), demand from j to h: $(trip2.demand)...\n")
                connection= ((trip1.bus_line_id, trip1.line_id,trip1.destination_stop_id),(trip2.bus_line_id, trip2.line_id,trip2.origin_stop_id))
                if !(connection in A_set)
                    push!(A_set, connection)
                    println("Added $connection to A_set.\n")
                else
                    println("Connection $connection already in A_set.\n")
                end
            else 
                #println("Connection from $s_i to $s_j not added, because on of the following is not true: i:$s_i > j:$s_j && k:$s_k < j:$s_j < i:$s_i && h:$s_h > j:$s_j or one of the demands is 0.\n")
            end
        end
    end
end

Added ((1, 1, 4), (1, 1, 2)) to A_set.

Connection ((1, 1, 4), (1, 1, 2)) already in A_set.

Added ((3, 1, 6), (3, 1, 2)) to A_set.

Added ((5, 3, 4), (5, 3, 2)) to A_set.

Connection ((5, 3, 4), (5, 3, 2)) already in A_set.



### Populating set V

In [23]:
V_set= Set{Tuple{Int, Int, Int}}()  # Set to hold all nodes
using OrderedCollections  
push!(V_set, depot)  # Add the depot D
for row in eachrow(demand)
    origin_tuple = (row.bus_line_id, row.line_id, row.origin_stop_id)
    destination_tuple = (row.bus_line_id, row.line_id, row.destination_stop_id)
    if !(origin_tuple in V_set)
        push!(V_set, origin_tuple)
    end
    if !(destination_tuple in V_set)
        push!(V_set, destination_tuple)
    end
end

V_set = sort(collect(V_set))
full_output = join(string.(V_set), "\n")
println("V_set:")
println(full_output) 

V_set:
(0, 0, 0)
(1, 1, 1)
(1, 1, 2)
(1, 1, 3)
(1, 1, 4)
(1, 2, 1)
(1, 2, 3)
(1, 2, 4)
(2, 1, 1)
(2, 1, 2)
(2, 3, 1)
(2, 3, 3)
(3, 1, 1)
(3, 1, 2)
(3, 1, 5)
(3, 1, 6)
(3, 2, 1)
(3, 2, 2)
(3, 2, 4)
(3, 2, 6)
(4, 2, 1)
(4, 2, 2)
(4, 4, 1)
(4, 4, 2)
(5, 1, 3)
(5, 1, 5)
(5, 3, 1)
(5, 3, 2)
(5, 3, 3)
(5, 3, 4)


# Model definition

## Initialize model instance

In [24]:
model= Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

## Initialize variables

In [25]:
@variable(model, x[connection in A_set, bus in unique(busses.bus_id)], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [((5, 3, 1), (5, 3, 4)), ((4, 4, 1), (4, 4, 2)), ((0, 0, 0), (2, 3, 1)), ((2, 1, 1), (2, 1, 2)), ((0, 0, 0), (1, 2, 3)), ((3, 1, 5), (0, 0, 0)), ((1, 1, 2), (1, 1, 3)), ((1, 1, 4), (2, 3, 1)), ((0, 0, 0), (3, 1, 1)), ((3, 2, 2), (0, 0, 0))  …  ((0, 0, 0), (4, 2, 1)), ((2, 1, 2), (5, 3, 2)), ((5, 3, 4), (5, 3, 2)), ((1, 1, 3), (4, 4, 1)), ((1, 1, 3), (5, 3, 2)), ((1, 1, 4), (3, 2, 4)), ((2, 1, 2), (3, 2, 4)), ((2, 3, 1), (2, 3, 3)), ((3, 2, 1), (3, 2, 2)), ((1, 1, 3), (3, 2, 4))]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
And data, a 74×10 Matrix{VariableRef}:
 x[((5, 3, 1), (5, 3, 4)),1]  …  x[((5, 3, 1), (5, 3, 4)),10]
 x[((4, 4, 1), (4, 4, 2)),1]     x[((4, 4, 1), (4, 4, 2)),10]
 x[((0, 0, 0), (2, 3, 1)),1]     x[((0, 0, 0), (2, 3, 1)),10]
 x[((2, 1, 1), (2, 1, 2)),1]     x[((2, 1, 1), (2, 1, 2)),10]
 x[((0, 0, 0), (1, 2, 3)),1]     x[((0, 0, 0), (1, 2, 3)),10]
 x[((3, 1, 5), (0, 0, 0)),1]  …  x[(

### Auf 0 fixieren

In [26]:
function violates_shift_or_break((i, j), bus, busses, nodes, demand, connections_dict)
    start_time_i = nodes[findfirst(x -> x.bus_line_id == i[1] && x.line_id==i[2] && x.stop_id==i[3], nodes)].start_time #start time of node i
    start_time_j = nodes[findfirst(x -> x.bus_line_id == j[1] && x.line_id==j[2] && x.stop_id==j[3], nodes)].start_time #start time of node j
    t_ij = connections_dict[(i,j)].travel_time #travel time from i to j
    shift_start = busses[findfirst(x -> x.bus_id == bus, eachrow(busses)),:shift_start] #shift start time
    shift_end = busses[findfirst(x -> x.bus_id == bus, eachrow(busses)), :shift_end] #shift end time
    if break_time == "break1"
        break_time_start = busses[findfirst(x -> x.bus_id == bus, eachrow(busses)), :break_start_1]  #break1 start time
        break_time_end= busses[findfirst(x -> x.bus_id == bus, eachrow(busses)), :break_end_1]  #break1 end time
    elseif break_time == "break2"
        break_time_start = busses[findfirst(x -> x.bus_id == bus, eachrow(busses)), :break_start_2]  #break2 start time
        break_time_end= busses[findfirst(x -> x.bus_id == bus, eachrow(busses)), :break_end_2]  #break2 end time
    else
        println("Warning: break_time is neither 'break1' nor 'break2'.")
    end
    

    #=
    println("Variables:")
    println("t_i: $t_i")
    println("t_j: $t_j")
    println("t_ij: $t_ij")
    println("shift_start: $shift_start")
    println("shift_end: $shift_end")
    println("break_time_start: $break_time_start")
    println("break2_start: $break2_start")
    =#

    # Fall (i): Start zu früh vom Depot
    if i == (0,0,0) && start_time_j - t_ij < shift_start
        reason= "Start zu früh vom Depot: start_j $start_time_j - t_ij $t_ij < shift_start $shift_start"
        return true, reason
    end

    # Fall (ii): Ankunft zu spät am Depot
    if j == (0,0,0) && start_time_i + t_ij > shift_end
        reason="Ankunft zu spät am Depot: start_i $start_time_i + t_ij $t_ij > shift_end $shift_end"
        return true, reason
    end

    # Fall (iii): Fahrt auf gleicher Linie – Pause betroffen
    if i[1]==j[1] && i[2]==j[2] # gleiche Linie und Tour
        if (break_time_start < start_time_i < break_time_end) ||        # Start des Trips liegt in der zweiten Pause
           (break_time_start < start_time_j < break_time_end) ||        # Ende des Trips liegt in der zweiten Pause
           (start_time_i < break_time_start && start_time_j > break_time_end)    # Trip überspannt die erste Pause
            reason="Fahrt auf gleicher Linie – Pause betroffen: start_i $start_time_i oder start_j $start_time_j in break1 ($break_time_start, $break_time_end)"
            return true, reason
        end
    else
        # Fall (iv): Linienwechsel – Pause muss dazwischen passen
        if (break_time_start < start_time_i < break_time_end) ||        # Start des Trips liegt in der zweiten Pause
           (break_time_start < start_time_j < break_time_end) ||        # Ende des Trips liegt in der zweiten Pause
           (start_time_j - start_time_i - t_ij < break_time_end - break_time_start && start_time_i < break_time_start && start_time_j > break_time_end)  # dazwischen reicht nicht
            reason="Linienwechsel – start_i: $start_time_i, start_j: $start_time_j, break1: ($break_time_start, $break_time_end)"
            return true, reason
        end
    end

    return false, "No Trips violate any breaks."
end


violates_shift_or_break (generic function with 1 method)

In [27]:
for bus in unique(busses.bus_id)
    for (i, j) in A_set
        violated, reason=violates_shift_or_break((i, j), bus, busses, nodes, demand, connections_dict)
        if violated
            fix(x[(i, j), bus], 0.0; force = true)
            println("Fixated: x[($i,$j), $bus] to 0 due to $reason.")
        end
    end
end


Fixated: x[((2, 3, 3),(0, 0, 0)), 1] to 0 due to Ankunft zu spät am Depot: start_i 61.92796532486692 + t_ij 16.701137685798535 > shift_end 70.
Fixated: x[((4, 4, 2),(0, 0, 0)), 1] to 0 due to Ankunft zu spät am Depot: start_i 61.02461938857064 + t_ij 14.580987620871227 > shift_end 70.
Fixated: x[((5, 3, 4),(0, 0, 0)), 1] to 0 due to Ankunft zu spät am Depot: start_i 62.399444310081115 + t_ij 14.098468001878787 > shift_end 70.
Fixated: x[((5, 3, 3),(0, 0, 0)), 1] to 0 due to Ankunft zu spät am Depot: start_i 54.97373222301499 + t_ij 16.716219668334105 > shift_end 70.
Fixated: x[((3, 1, 5),(0, 0, 0)), 2] to 0 due to Linienwechsel – start_i: 39.98408309247563, start_j: 0.0, break1: (35, 40).
Fixated: x[((0, 0, 0),(3, 1, 1)), 2] to 0 due to Start zu früh vom Depot: start_j 16.0 - t_ij 12.473652231804444 < shift_start 5.
Fixated: x[((3, 2, 2),(0, 0, 0)), 2] to 0 due to Linienwechsel – start_i: 37.426601146205606, start_j: 0.0, break1: (35, 40).
Fixated: x[((4, 2, 2),(5, 3, 1)), 2] to 0 due 

## Objective function

In [28]:
@objective(model, Min, sum(x[connection, bus] for bus in unique(busses.bus_id), connection in A_set if connection[1] == (0, 0, 0)))

x[((0, 0, 0), (2, 3, 1)),1] + x[((0, 0, 0), (1, 2, 3)),1] + x[((0, 0, 0), (3, 1, 1)),1] + x[((0, 0, 0), (1, 1, 2)),1] + x[((0, 0, 0), (5, 3, 1)),1] + x[((0, 0, 0), (5, 1, 3)),1] + x[((0, 0, 0), (2, 1, 1)),1] + x[((0, 0, 0), (1, 2, 1)),1] + x[((0, 0, 0), (3, 1, 2)),1] + x[((0, 0, 0), (4, 4, 1)),1] + x[((0, 0, 0), (5, 3, 2)),1] + x[((0, 0, 0), (3, 2, 1)),1] + x[((0, 0, 0), (3, 2, 4)),1] + x[((0, 0, 0), (1, 1, 1)),1] + x[((0, 0, 0), (4, 2, 1)),1] + x[((0, 0, 0), (2, 3, 1)),2] + x[((0, 0, 0), (1, 2, 3)),2] + x[((0, 0, 0), (3, 1, 1)),2] + x[((0, 0, 0), (1, 1, 2)),2] + x[((0, 0, 0), (5, 3, 1)),2] + x[((0, 0, 0), (5, 1, 3)),2] + x[((0, 0, 0), (2, 1, 1)),2] + x[((0, 0, 0), (1, 2, 1)),2] + x[((0, 0, 0), (3, 1, 2)),2] + x[((0, 0, 0), (4, 4, 1)),2] + x[((0, 0, 0), (5, 3, 2)),2] + x[((0, 0, 0), (3, 2, 1)),2] + x[((0, 0, 0), (3, 2, 4)),2] + x[((0, 0, 0), (1, 1, 1)),2] + x[((0, 0, 0), (4, 2, 1)),2] + [[...90 terms omitted...]] + x[((0, 0, 0), (2, 3, 1)),9] + x[((0, 0, 0), (1, 2, 3)),9] + x[((0, 0, 0

## Constraints

### Flusserhaltung

In [29]:
@constraint(model, 
    flow_conservation[node in V_set, bus in unique(busses.bus_id)],
    sum(x[connection,bus] for connection in A_set if connection[2] == node) - 
    sum(x[connection,bus] for connection in A_set if connection[1] == node) == 0
)


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [(0, 0, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 2, 1), (1, 2, 3), (1, 2, 4), (2, 1, 1), (2, 1, 2)  …  (4, 2, 1), (4, 2, 2), (4, 4, 1), (4, 4, 2), (5, 1, 3), (5, 1, 5), (5, 3, 1), (5, 3, 2), (5, 3, 3), (5, 3, 4)]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
And data, a 30×10 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 flow_conservation[(0, 0, 0),1] : -x[((0, 0, 0), (2, 3, 1)),1] - x[((0, 0, 0), (1, 2, 3)),1] + x[((3, 1, 5), (0, 0, 0)),1] - x[((0, 0, 0), (3, 1, 1)),1] + x[((3, 2, 2), (0, 0, 0)),1] + x[((5, 1, 5), (0, 0, 0)),1] - x[((0, 0, 0), (1, 1, 2)),1] + x[((2, 3, 3), (0, 0, 0)),1] - x[((0, 0, 0), (5, 3, 1)),1] - x[((0, 0, 0), (5, 1, 3)

### All customer trips are served

In [30]:
@constraint(model,[connection in customer_trips],
    sum(x[connection,bus] for bus in unique(busses.bus_id)) == 1
)
#ich habe hier die erneute Prüfung nach dem positiven demand weggelassen, weil nur Verbindungen in A_set aufgenommen werden, die einen positiven demand haben

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [((1, 1, 1), (1, 1, 4)), ((1, 1, 2), (1, 1, 3)), ((1, 1, 2), (1, 1, 4)), ((1, 2, 3), (1, 2, 4)), ((1, 2, 1), (1, 2, 3)), ((2, 1, 1), (2, 1, 2)), ((2, 3, 1), (2, 3, 3)), ((3, 1, 1), (3, 1, 6)), ((3, 1, 2), (3, 1, 5)), ((3, 2, 1), (3, 2, 2)), ((3, 2, 4), (3, 2, 6)), ((4, 2, 1), (4, 2, 2)), ((4, 4, 1), (4, 4, 2)), ((5, 1, 3), (5, 1, 5)), ((5, 3, 2), (5, 3, 4)), ((5, 3, 2), (5, 3, 3)), ((5, 3, 1), (5, 3, 4))]
And data, a 17-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[((1, 1, 1), (1, 1, 4)),1] + x[((1, 1, 1), (1, 1, 4)),2] + x[((1, 1, 1), (1, 1, 4)),3] + x[((1, 1, 1), (1, 1, 4)),4] + x[((1, 1, 1), (1, 1, 4)),5] + x[((1, 1, 1), (1, 1, 4)),6] + x[((1,

### Each bus only once

In [31]:
@constraint(model, bus_usage[bus in unique(busses.bus_id)], sum(x[connection, bus] for connection in A_set if connection[1] == (0, 0, 0))<=1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 bus_usage[1] : x[((0, 0, 0), (2, 3, 1)),1] + x[((0, 0, 0), (1, 2, 3)),1] + x[((0, 0, 0), (3, 1, 1)),1] + x[((0, 0, 0), (1, 1, 2)),1] + x[((0, 0, 0), (5, 3, 1)),1] + x[((0, 0, 0), (5, 1, 3)),1] + x[((0, 0, 0), (2, 1, 1)),1] + x[((0, 0, 0), (1, 2, 1)),1] + x[((0, 0, 0), (3, 1, 2)),1] + x[((0, 0, 0), (4, 4, 1)),1] + x[((0, 0, 0), (5, 3, 2)),1] + x[((0, 0, 0), (3, 2, 1)),1] + x[((0, 0, 0), (3, 2, 4)),1] + x[((0, 0, 0), (1, 1, 1)),1] + x[((0, 0, 0), (4, 2, 1)),1] ≤ 1
 bus_usage[2] : x[((0, 0, 0), (2, 3, 1)),2] + x[((0, 0, 0), (1, 2, 3)),2] + x[((0, 0, 0), (3, 

### Backard connection with line change

In [32]:
for k in unique(busses.bus_id)
    for connection1 in A_set
        (i,j)=connection1
        #println("Adding constraints for connection $connection and bus $k...")
        #=Checking if i and j are either
            a)stops on the same bus-line, but different tours
            b)stops on different bus lines =#
        if (i[1] == j[1] && i[2] != j[2]) || i[1] != j[1]
            for connection2 in A_set
                (h, h_)=connection2
                if h[1] == h_[1] == i[1] && h[2] == h_[2] == i[2]  
                    #println("  h:$h and h_:$h_ are on the same bus line and tour as i:$i...")
                    #println("     Checking if h[3] <= i[3] < h_[3]...")
                    if h[3] <= i[3] < h_[3]
                        idx = findfirst(row -> row.bus_line_id == h[1] && row.line_id == h[2] && row.origin_stop_id == h[3] && row.destination_stop_id == h_[3], eachrow(demand))
                        has_demand = !isnothing(idx) && demand[idx, :demand] > 0    
                        #println("      Checking if demand is positive and if time constraint holds..." )
                        if has_demand && nodes[findfirst(n -> n.bus_line_id == h_[1] && n.line_id == h_[2] && n.stop_id == h_[3], nodes)].start_time + connections_dict[(i, j)].travel_time > nodes[findfirst(n -> n.bus_line_id == j[1] && n.line_id == j[2] && n.stop_id == j[3], nodes)].start_time
                            @constraint(model, x[(i, j), k] <= 1 - x[(h, h_), k])
                            println("        Constraint added for bus $k: x[($i, $j), $k] <= 1 - x[($h, $h_), $k]")
                        end
                    end
                end
            end
        end
    end
end


        Constraint added for bus 1: x[((3, 1, 5), (0, 0, 0)), 1] <= 1 - x[((3, 1, 1), (3, 1, 6)), 1]
        Constraint added for bus 1: x[((3, 1, 5), (3, 2, 4)), 1] <= 1 - x[((3, 1, 1), (3, 1, 6)), 1]
        Constraint added for bus 1: x[((1, 1, 3), (5, 3, 1)), 1] <= 1 - x[((1, 1, 2), (1, 1, 4)), 1]
        Constraint added for bus 1: x[((1, 1, 3), (5, 3, 1)), 1] <= 1 - x[((1, 1, 1), (1, 1, 4)), 1]
        Constraint added for bus 1: x[((1, 1, 3), (0, 0, 0)), 1] <= 1 - x[((1, 1, 2), (1, 1, 4)), 1]
        Constraint added for bus 1: x[((1, 1, 3), (0, 0, 0)), 1] <= 1 - x[((1, 1, 1), (1, 1, 4)), 1]
        Constraint added for bus 1: x[((5, 3, 3), (0, 0, 0)), 1] <= 1 - x[((5, 3, 1), (5, 3, 4)), 1]
        Constraint added for bus 1: x[((5, 3, 3), (0, 0, 0)), 1] <= 1 - x[((5, 3, 2), (5, 3, 4)), 1]
        Constraint added for bus 1: x[((1, 2, 3), (0, 0, 0)), 1] <= 1 - x[((1, 2, 3), (1, 2, 4)), 1]
        Constraint added for bus 1: x[((1, 1, 3), (3, 2, 4)), 1] <= 1 - x[((1, 1, 2), (1, 1

### Capacity constraint

In [33]:
@constraint(model, [bus in unique(busses.bus_id), node in V_set ; node != (0,0,0)], 
    sum(demand[findfirst(row -> row.bus_line_id == connection[1][1] && row.line_id == connection[1][2] && row.origin_stop_id == connection[1][3] && row.destination_stop_id == connection[2][3], eachrow(demand)),:demand] * x[connection,bus] for connection in customer_trips if connection[1][1]==connection[2][1] && connection[1][2]==connection[2][2] && connection[1][3]<= node[3] < connection[2][3]) <= busses[findfirst(row -> row.bus_id == bus, eachrow(busses)), :capacity]
    
)

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 2, Tuple{Int64, Tuple{Int64, Int64, Int64}}} with 290 entries:
  [1, (1, 1, 1) ]  =  3 x[((5, 3, 1), (5, 3, 4)),1] + 2 x[((4, 4, 1), (4, 4, 2)),1] + 4 x[((2, 1, 1), (2, 1, 2)),1] + 2 x[((3, 1, 1), (3, 1, 6)),1] + x[((4, 2, 1), (4, 2, 2)),1] + x[((1, 1, 1), (1, 1, 4)),1] + 2 x[((1, 2, 1), (1, 2, 3)),1] + x[((2, 3, 1), (2, 3, 3)),1] + x[((3, 2, 1), (3, 2, 2)),1] ≤ 10
  [1, (1, 1, 2) ]  =  3 x[((5, 3, 1), (5, 3, 4)),1] + 2 x[((1, 1, 2), (1, 1, 3)),1] + x[((3, 1, 2), (3, 1, 5)),1] + x[((1, 1, 2), (1, 1, 4)),1] + 2 x[((3, 1, 1), (3, 1, 6)),1] + x[((1, 1, 1), (1, 1, 4)),1] + x[((5, 3, 2), (5, 3, 3)),1] + 2 x[((1, 2, 1), (1, 2, 3)),1] + x[((5, 3, 2), (5, 3, 4)),1] + x[((2, 3, 1), (2, 3, 3)),1] ≤ 10
  [1, (1, 1, 3) ]  =  3 x[((5, 3, 1), (5, 3, 4)),1] + x[((3, 1, 2), (3, 1, 5)),1] + x[((1, 1, 2), (1, 1, 4)),1] + 

# Solve model

In [34]:
optimize!(model)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+00]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
244 rows, 314 cols, 1389 nonzeros  0s
202 rows, 248 cols, 1308 nonzeros  0s
167 rows, 214 cols, 1061 nonzeros  0s
108 rows, 196 cols, 712 nonzeros  0s
98 rows, 152 cols, 591 nonzeros  0s
Presolve: Infeasible

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0    -80      0         0     0.0s

Solving report
  Status            Infeasible
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  Solution status   -
  Timing            0.01 (total)
                    0.00

## Termination status

In [35]:
println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))
println("Dual status: ", dual_status(model))

if termination_status(model) == MOI.OPTIMAL
    println("Optimale Lösung gefunden!")
    println("Zielfunktionswert: ", objective_value(model))
else
    println("Kein Optimum – Solver-Status: ", termination_status(model))
end
    

Termination status: INFEASIBLE
Primal status: NO_SOLUTION
Dual status: NO_SOLUTION
Kein Optimum – Solver-Status: INFEASIBLE


## Active Connections

In [36]:
active_connections = []
for connection in A_set , bus in unique(busses.bus_id)
    if value(x[connection,bus]) > 1e-6
        push!(active_connections, connection)
    end
end

#= sort(active_connections, by = x -> x[1][1][1])
full_output = join(string.(active_connections), "\n")
println(full_output) =#

MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

## Chosen routes

In [37]:
function try_path(current_node, route_temp, path, visited)
    #println("➡️ Suche Weiterführung ab: ", current_node)
    next_edges = filter(conn -> conn[1] == current_node && !(conn in visited), route_temp)
    #println("   🔎 Gefundene Anschlusskanten: ", next_edges)

    if isempty(next_edges)
        if current_node == (0, 0, 0)
            #println("Pfad endet korrekt im Depot.")
            return copy(path)
        else
            #println("Sackgasse erreicht bei ", current_node)
            return nothing
        end
    end

    for conn in next_edges
        push!(path, conn)
        push!(visited, conn)
        #println("Kante hinzugefügt: ", conn)
        result = try_path(conn[2], route_temp, path, visited)
        if result != nothing
            return result
        else
            #println("Backtracking von ", conn)
            pop!(path)
            delete!(visited, conn)
        end
    end
    return nothing
end

function find_complete_path(route_temp)
    #println("🔍 Suche Pfad in: ", route_temp)
    start_edges = filter(conn -> conn[1] == (0, 0, 0), route_temp)
    if length(start_edges) != 1
        #println("Kein eindeutiger Startknoten gefunden.")
        return [], false
    end

    start_conn = start_edges[1]
    #println("Starte Pfadsuche mit: ", start_conn)
    path = [start_conn]
    visited = Set([start_conn])
    result = try_path(start_conn[2], route_temp, path, visited)

    return result === nothing ? ([], false) : (result, true)
end

# Hauptfunktion: Routen sortieren
routes_dict = Dict{Int, Vector{Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}}}()

for bus in unique(busses.bus_id)
    route_temp = [conn for conn in active_connections if value(x[conn, bus]) > 1e-6]

    if !isempty(route_temp)
        full_path, ok = find_complete_path(route_temp)
        if ok
            #println("Vollständiger Pfad für Bus $bus: ", full_path)
            routes_dict[bus] = full_path
        else
            #println("Kein gültiger Pfad für Bus $bus gefunden.")
            routes_dict[bus] = []
        end
    else
        #println("Keine aktiven Verbindungen für Bus $bus.")
        routes_dict[bus] = []
    end
end



In [38]:
for key in 1:length(keys(routes_dict))
    println("Route für Bus $key:", routes_dict[key])
end

Route für Bus 1:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 2:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 3:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 4:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 5:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 6:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 7:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 8:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 9:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]
Route für Bus 10:Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}[]


In [39]:
function get_served_customer_trips(routes_dict, customer_trips)
    served_trips = Dict{Int, Vector{Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}}}()

    for (bus, route) in routes_dict
        served = Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}[]  # leere Liste für diesen Bus

        for cust_trip in customer_trips
            if cust_trip in route
                push!(served, cust_trip)
            end
        end

        served_trips[bus] = served
    end

    return served_trips
end


get_served_customer_trips (generic function with 1 method)

In [40]:
served_trips = get_served_customer_trips(routes_dict, customer_trips)

for (bus, trips) in served_trips
    println("Bus $bus bedient folgende Customer Trips:")
    if trips == []
        println("    Keine Trips bedient.")
        continue
    else
        for t in trips
            println("    $t")
        end
    end
end

Bus 5 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 4 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 6 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 7 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 2 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 10 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 9 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 8 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 3 bedient folgende Customer Trips:
    Keine Trips bedient.
Bus 1 bedient folgende Customer Trips:
    Keine Trips bedient.


# Visualization

In [41]:
# Store full node struct as value for lookup
active_nodes_dict = Dict{Tuple{Int, Int, Int}, typeof(nodes[1])}()
for connection in active_connections
    origin,goal=connection
    a,b,c = origin
    d,e,f = goal
    node=findfirst(x -> x.bus_line_id == a && x.line_id == b && x.stop_id == c, nodes)
    if !(node in keys(active_nodes_dict))
        active_nodes_dict[origin] = nodes[node]
    end
    node=findfirst(x -> x.bus_line_id == d && x.line_id == e && x.stop_id == f, nodes)
    if !(node in keys(active_nodes_dict))
        active_nodes_dict[goal] = nodes[node]
    end
end
sort(collect(active_nodes_dict))

Pair{Tuple{Int64, Int64, Int64}, @NamedTuple{bus_line_id::Int64, line_id::Int64, stop_id::Int64, coord_x::Float64, coord_y::Float64, start_time::Float64}}[]

## Plot

### Functional

In [42]:
using Dates

# Farben für Busse (ggf. erweitern)
colors = ["red", "blue", "green", "orange", "purple", "brown", "cyan", "magenta", "black", "gray"]

# 1. Nur besuchte Knoten extrahieren
visited_nodes = Set{Tuple{Int, Int, Int}}()
for conns in values(routes_dict)
    for (from, to) in conns
        push!(visited_nodes, from)
        push!(visited_nodes, to)
    end
end

# 2. Nur besuchte Knoten anzeigen
filtered_nodes = filter(n -> (n.bus_line_id, n.line_id, n.stop_id) in visited_nodes, nodes)

xs = [n.coord_x for n in filtered_nodes]
ys = [n.coord_y for n in filtered_nodes]
zs = [n.start_time for n in filtered_nodes]
labels = [string("(", n.bus_line_id, ", ", n.line_id, ", ", n.stop_id, ")\n", "t=", round(n.start_time, digits=1)) for n in filtered_nodes]

trace_nodes = scatter3d(
    x=xs, y=ys, z=zs,
    mode="markers",
    marker=attr(size=3, color="blue"),
    text=labels,
    hoverinfo="text",
    name="Visited Stops"
)

# 3. Lookup-Tabelle für alle Knoten
nodes_dict = Dict((n.bus_line_id, n.line_id, n.stop_id) => n for n in nodes)

# 4. Linien je Bus visualisieren
traces_edges = []

shown_main_legends = Set{Int}()
shown_transfer_legends = Set{Int}()
shown_reverse_legends = Set{Int}()

for (i, route) in pairs(routes_dict)
    color = colors[mod1(i, length(colors))]

    for (from, to) in route
        if from[1:2] == to[1:2]
            # gleiche Linie
            stop_from = from[3]
            stop_to = to[3]

            if stop_from == stop_to
                continue
            end

            if stop_from < stop_to
                # Vorwärtsfahrt mit Zwischenknoten
                stops_range = stop_from:stop_to
                intermediate_stops = [(from[1], from[2], s) for s in stops_range]
                path_nodes = [nodes_dict[s] for s in intermediate_stops if s in keys(nodes_dict)]

                push!(traces_edges, scatter3d(
                    x = [n.coord_x for n in path_nodes],
                    y = [n.coord_y for n in path_nodes],
                    z = [n.start_time for n in path_nodes],
                    mode = "lines",
                    line = attr(width=3, color=color),
                    hoverinfo = "none",
                    showlegend = !(i in shown_main_legends),
                    name = "Bus $i"
                ))
                push!(shown_main_legends, i)
            else
                # Rückfahrt → direkte Linie
                x_vals = [nodes_dict[from].coord_x, nodes_dict[to].coord_x]
                y_vals = [nodes_dict[from].coord_y, nodes_dict[to].coord_y]
                z_vals = [nodes_dict[from].start_time, nodes_dict[to].start_time]

                push!(traces_edges, scatter3d(
                    x = x_vals, y = y_vals, z = z_vals,
                    mode = "lines",
                    line = attr(width=3, color=color, dash="dash"),
                    hoverinfo = "none",
                    showlegend = !(i in shown_reverse_legends),
                    name = "Rückfahrt Bus $i"
                ))
                push!(shown_reverse_legends, i)
            end
        else
            # Linienwechsel oder Depotfahrt
            x_vals = [nodes_dict[from].coord_x, nodes_dict[to].coord_x]
            y_vals = [nodes_dict[from].coord_y, nodes_dict[to].coord_y]
            z_vals = [nodes_dict[from].start_time, nodes_dict[to].start_time]

            push!(traces_edges, scatter3d(
                x = x_vals, y = y_vals, z = z_vals,
                mode = "lines",
                line = attr(width=2, color=color, dash="dot"),
                hoverinfo = "none",
                showlegend = !(i in shown_transfer_legends),
                name = "Transfer Bus $i"
            ))
            push!(shown_transfer_legends, i)
        end
    end
end

# 5. Layout & Export
layout = Layout(
    title = "Bus Tours (Visited Stops & Real Paths)",
    scene = attr(
        xaxis=attr(title="X"),
        yaxis=attr(title="Y"),
        zaxis=attr(title="Start Time")
    ),
    showlegend=true
)

plot_obj = Plot([trace_nodes; traces_edges...], layout)
timestamp = Dates.format(now(), "yyyymmdd_HH-MM")
savefig(plot_obj, "/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_setting_3_$timestamp.html")


"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_setting_3_20250805_08-18.html"

# Error analysis

In [43]:
conflict = compute_conflict!(model)

ArgumentError: ArgumentError: The optimizer HiGHS.Optimizer does not support `compute_conflict!`

In [44]:
value.(x)  # alle Variablenwerte


MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [45]:
for c in all_constraints(model)
    println("Constraint: ", c)
    println("   Value: ", value(c))
    println("   Dual: ", dual(c))
end


UndefKeywordError: UndefKeywordError: keyword argument `include_variable_in_set_constraints` not assigned

In [46]:
A_set = sort(collect(A_set))
full_output = join(string.(A_set), "\n")
println("A_set:")
println(full_output) 

A_set:
((0, 0, 0), (1, 1, 1))
((0, 0, 0), (1, 1, 2))
((0, 0, 0), (1, 2, 1))
((0, 0, 0), (1, 2, 3))
((0, 0, 0), (2, 1, 1))
((0, 0, 0), (2, 3, 1))
((0, 0, 0), (3, 1, 1))
((0, 0, 0), (3, 1, 2))
((0, 0, 0), (3, 2, 1))
((0, 0, 0), (3, 2, 4))
((0, 0, 0), (4, 2, 1))
((0, 0, 0), (4, 4, 1))
((0, 0, 0), (5, 1, 3))
((0, 0, 0), (5, 3, 1))
((0, 0, 0), (5, 3, 2))
((1, 1, 1), (1, 1, 4))
((1, 1, 2), (1, 1, 3))
((1, 1, 2), (1, 1, 4))
((1, 1, 3), (0, 0, 0))
((1, 1, 3), (1, 2, 1))
((1, 1, 3), (1, 2, 3))
((1, 1, 3), (2, 3, 1))
((1, 1, 3), (3, 2, 4))
((1, 1, 3), (4, 4, 1))
((1, 1, 3), (5, 3, 1))
((1, 1, 3), (5, 3, 2))
((1, 1, 4), (0, 0, 0))
((1, 1, 4), (1, 1, 2))
((1, 1, 4), (1, 2, 3))
((1, 1, 4), (2, 3, 1))
((1, 1, 4), (3, 2, 4))
((1, 1, 4), (4, 4, 1))
((1, 2, 1), (1, 2, 3))
((1, 2, 3), (0, 0, 0))
((1, 2, 3), (1, 2, 4))
((1, 2, 4), (0, 0, 0))
((2, 1, 1), (2, 1, 2))
((2, 1, 2), (0, 0, 0))
((2, 1, 2), (1, 2, 3))
((2, 1, 2), (2, 3, 1))
((2, 1, 2), (3, 2, 4))
((2, 1, 2), (4, 4, 1))
((2, 1, 2), (5, 3, 1))
((2,

In [47]:
for (i, j) in A_set
    if i != depot && j != depot
        t_end = connections_dict[(i,j)].end_time
        t_start = nodes[findfirst(n -> n.bus_line_id == j[1] && n.line_id == j[2] && n.stop_id == j[3], nodes)].start_time
        if t_end > t_start
            println("Zeitverletzung: $(i) → $(j) endet $t_end aber Start ist $t_start")
        end
    end
end


Zeitverletzung: (1, 1, 4) → (1, 1, 2) endet 41.93380653841311 aber Start ist 22.379663841806234
Zeitverletzung: (3, 1, 6) → (3, 1, 2) endet 65.07981104903452 aber Start ist 21.426601146205606
Zeitverletzung: (5, 3, 4) → (5, 3, 2) endet 70.25188342159291 aber Start ist 51.42560595694158


In [48]:
for (i,j) in A_set
    if !haskey(connections_dict, (i,j))
        println("Fehlende Verbindung in connections_dict für: ($i,$j)")
    end
end


In [49]:
missing_trips_in_connections_dict

1-element Vector{Any}:
 ((1, 2, 3), (1, 2, 3))

In [50]:
counter=0
for i in customer_trips
    from,to=i
    if (depot,from) in A_set
        if i in A_set
            if (to,depot) in A_set
                println("Complete route for customer trip $i in A_set.")
                counter+=1
                
            else
                println("Connection from depot to $from in A_set, but connection from $to to depot not in A_set.")
            end
        else
            println("Connection from depot to $from in A_set, but $i is not in A__set.")
        end
    else
        println("Connection from depot to $from not in A_set.")
    end
end
count_customer_trips = length(customer_trips)
if counter == count_customer_trips
    println("All $count_customer_trips customer trips are covered by the routes in A_set.")
else
    println("Not all customer trips are covered by the routes in A_set.")
end

Complete route for customer trip ((1, 1, 1), (1, 1, 4)) in A_set.
Complete route for customer trip ((1, 1, 2), (1, 1, 3)) in A_set.
Complete route for customer trip ((1, 1, 2), (1, 1, 4)) in A_set.
Complete route for customer trip ((1, 2, 3), (1, 2, 4)) in A_set.
Complete route for customer trip ((1, 2, 1), (1, 2, 3)) in A_set.
Complete route for customer trip ((2, 1, 1), (2, 1, 2)) in A_set.
Complete route for customer trip ((2, 3, 1), (2, 3, 3)) in A_set.
Complete route for customer trip ((3, 1, 1), (3, 1, 6)) in A_set.
Complete route for customer trip ((3, 1, 2), (3, 1, 5)) in A_set.
Complete route for customer trip ((3, 2, 1), (3, 2, 2)) in A_set.
Complete route for customer trip ((3, 2, 4), (3, 2, 6)) in A_set.
Complete route for customer trip ((4, 2, 1), (4, 2, 2)) in A_set.
Complete route for customer trip ((4, 4, 1), (4, 4, 2)) in A_set.
Complete route for customer trip ((5, 1, 3), (5, 1, 5)) in A_set.
Complete route for customer trip ((5, 3, 2), (5, 3, 4)) in A_set.
Complete r

In [51]:
V_set = sort(collect(V_set))
full_output = join(string.(V_set), "\n")
println("V_set:")
println(full_output) 

V_set:
(0, 0, 0)
(1, 1, 1)
(1, 1, 2)
(1, 1, 3)
(1, 1, 4)
(1, 2, 1)
(1, 2, 3)
(1, 2, 4)
(2, 1, 1)
(2, 1, 2)
(2, 3, 1)
(2, 3, 3)
(3, 1, 1)
(3, 1, 2)
(3, 1, 5)
(3, 1, 6)
(3, 2, 1)
(3, 2, 2)
(3, 2, 4)
(3, 2, 6)
(4, 2, 1)
(4, 2, 2)
(4, 4, 1)
(4, 4, 2)
(5, 1, 3)
(5, 1, 5)
(5, 3, 1)
(5, 3, 2)
(5, 3, 3)
(5, 3, 4)
